In [1]:
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits
from astropy.wcs import WCS
from astropy.utils.data import download_file
from astropy.nddata import Cutout2D
from pathlib import Path
import os 
import re
from astropy.coordinates import SkyCoord
import zipfile
from PIL import Image
import io
from astropy.stats import sigma_clipped_stats    

## Convert file names to lists for all 300 tiles

#### 1. Specify directories and prefices

In [2]:
# Paths to mosaics and catalogues directories
mosaics_path = Path('/data1/euclid_data/mosaics')
cat_path = Path('/data1/euclid_data/catalogues')

# Pattern to find the 'TILE' identifier followed by digits
tile_pattern = re.compile(r'TILE\d+')

# Prefix to filter catalog files (modify as needed)
prefix = "EUC_MER_FINAL-CAT_"

#### 2. Generate list for final catalogue files

In [3]:
# Get an ordered list of TILE identifiers from cat_path files that start with the prefix
cat_list = [] 
for item in cat_path.iterdir():
    if item.name.startswith(prefix) and item.is_file():
        cat_match = tile_pattern.search(str(item))
        if cat_match:
            cat_list.append((cat_match.group(), item))  # (TILE_id, cat_file)

#### 3. Generate list for masaics files

In [4]:
# Sort mosaics to match the order of cat_list
matching_mosaics = []
for tile_id, cat_file in cat_list:
    for item in mosaics_path.iterdir():
        if item.is_file():
            mosaic_match = tile_pattern.search(str(item))
            if mosaic_match and mosaic_match.group() == tile_id:
                matching_mosaics.append(item)
                break  # Stop searching once the matching mosaic is found

#### 4. Generate list for morphology catalogues

In [5]:
# Sort morphology catalogues to match the order of cat_list 
morph_prefix = 'EUC_MER_FINAL-MORPH-CAT'
matching_morph_cat = []
for tile_id, cat_file in cat_list:
    for item in cat_path.iterdir():
        if item.is_file() and (morph_prefix in str(item)):
            morph_match = tile_pattern.search(str(item))
            if morph_match and morph_match.group() == tile_id: 
                matching_morph_cat.append(item)
                break  # Stop searching once the morph cat is found

# Print results
print(len(matching_mosaics)) # check integrity 

344


## Adjust the number of tiles used

In [6]:
tile_num = 344 # adjust the number of catalogues used 

## Building Filters

#### 1. Generate a subset without star-like features and too much brightness

In [7]:
def ujy_to_mag(ujy):
    jy = ujy * 10 ** -6
    mag = -2.5 * np.log10(jy) + 8.90
    return mag

In [8]:
objectId_list1 = []
subset_list = []
for i in range(0,tile_num): # Adjust the number of catalogue that will be used 
    
    with fits.open(cat_list[i][1], memmap=True) as hdul:
        tab = hdul[1].data
        flux_positive = tab['FLUX_DETECTION_TOTAL'] > 0
        tab_filtered = tab[flux_positive]        
        mag = ujy_to_mag(tab_filtered['FLUX_DETECTION_TOTAL'])
        subset = tab_filtered[(mag < 22) & (tab_filtered['POINT_LIKE_FLAG'] != 1)] 
        subset = subset[subset['DET_QUALITY_FLAG'] < 1]
    
    subset_list.append((subset['OBJECT_ID'],subset['RIGHT_ASCENSION'],subset['DECLINATION'])) # for all tiles 
    
    objectId_list1.append(subset['OBJECT_ID'])

    if (i+1)%43 == 0:
        print(f'Completed: {(i+1)}/344')

Completed: 43/344
Completed: 86/344
Completed: 129/344
Completed: 172/344
Completed: 215/344
Completed: 258/344
Completed: 301/344
Completed: 344/344


#### 2. Generate another subset without spiral arms

In [9]:
objectId_list2 = []
for i in range(0,tile_num): # Adjust the number of catalogue that will be used 
    
    with fits.open(matching_morph_cat[i], memmap=True) as hdul:
        tab = hdul[1].data
        tab_new = tab[~((tab['HAS_SPIRAL_ARMS_YES']+(100-tab['HAS_SPIRAL_ARMS_NO']))/200 > 0.5)]

    objectId_list2.append(tab_new['OBJECT_ID'])
    if (i+1)%43 == 0:
        print(f'Completed: {(i+1)}/344')

Completed: 43/344
Completed: 86/344
Completed: 129/344
Completed: 172/344
Completed: 215/344
Completed: 258/344
Completed: 301/344
Completed: 344/344


#### 3. Extract the overlapping `OBJECT_ID`s from the two lists

In [10]:
merged_objectId_list = [] # for all tiles 
for i in range(0,tile_num): # iterate each tile  
    single_objectId_list = list(set(objectId_list1[i]).intersection(objectId_list2[i]))
    merged_objectId_list.append(single_objectId_list)
    
    if (i+1)%43 == 0:
        print(f'Completed: {(i+1)}/344')

Completed: 43/344
Completed: 86/344
Completed: 129/344
Completed: 172/344
Completed: 215/344
Completed: 258/344
Completed: 301/344
Completed: 344/344


## Cutout Generation

#### 1. Generate RA and DEC Lists for Real World Coordinates by matching `OBJECT_ID`

In [11]:
all_obj_ra_dec = []
for i, single_objectId_list in enumerate(merged_objectId_list): # objectIds for a single tile 
    # for j, single_subset in enumerate(subset_list): # subset of a single tile 
    obj_ra_dec = []
    for objectId in single_objectId_list: # for each object 
    #     if objectId in single_subset[0]:
        ra = subset_list[i][1][np.where(subset_list[i][0] == objectId)[0][0]]
        dec = subset_list[i][2][np.where(subset_list[i][0] == objectId)[0][0]]
        obj_ra_dec.append((objectId, ra, dec))
    all_obj_ra_dec.append(obj_ra_dec)

    if (i+1)%43 == 0:
        print(f'Completed: {(i+1)}/344')

Completed: 43/344
Completed: 86/344
Completed: 129/344
Completed: 172/344
Completed: 215/344
Completed: 258/344
Completed: 301/344
Completed: 344/344


#### 2. Set Cutout Dimension

In [12]:
cutout_size = (96,96)

#### 3. Produce cutouts and save them in zip file

In [13]:
def is_valid_image(cutout_data):

    max_val = np.max(cutout_data)  
    std_val = np.std(cutout_data)
    
    # Check if image is too dark
    if (max_val < 0.05):
        return False
    if (max_val < 0.06) & (std_val <= 0.01):
        return False 
    return True

In [14]:
def not_defected(img):
    img = np.array(img)
    region = img[46:50,46:50]
    is_same = len(np.unique(region)) == 1
    is_dark = np.all(region < 150)
    if is_same and is_dark:
        return False 
    else:
        return True 

In [ ]:
# Assuming fits_path is a valid FITS file in the matching_mosaics list
fits_path_list = matching_mosaics[0:tile_num] # select mosaics 

output_dir = '/home/xinmeix/ml_cutouts/final_cutout'  # Update this path to your desired directory
# zip_filename = os.path.join(output_dir, 'ml_cutouts_1perc.zip')
cutout_files = []
all_cutout_files = []

for i, fits_path in enumerate(fits_path_list): # iterate through every mosaic file 
# Open the FITS file
    with fits.open(fits_path, memmap=True) as hdul: # for each tile 
        # Create WCS object from the FITS header
        wcs = WCS(hdul[0].header)
    
        # Convert RA and DEC to a SkyCoord object
        onetile_ra_list = np.array([t[1] for t in all_obj_ra_dec[i]]) # ra data for one tile in np array format
        onetile_dec_list = np.array([t[2] for t in all_obj_ra_dec[i]]) # dec data for one tile in np array format
        onetile_obj_list = [t[0] for t in all_obj_ra_dec[i]]
        
        sky_coord = SkyCoord(onetile_ra_list, onetile_dec_list, unit='deg', frame='icrs') # for each tile 
    
        # Convert RA, DEC to pixel coordinates using WCS
        pixel_coords = wcs.world_to_pixel(sky_coord)

        # Loop through the pixel coordinates and generate cutouts
        for j, (x, y) in enumerate(zip(pixel_coords[0], pixel_coords[1])): # for each object 
            x, y = int(np.round(x)), int(np.round(y))
            position = (x, y)
            
            # Create the cutout using Cutout2D
            cutout_region = Cutout2D(hdul[0].data, position, size=cutout_size, wcs=wcs)
            cutout_data = cutout_region.data

            if is_valid_image(cutout_data):
        
                # Ensure the index is within bounds of the merged objectId list 
                if j < len(merged_objectId_list[i]):
                    # Prepare the filename for each cutout image using the OBJECT_ID
                    cutout_filename = f"{str(matching_mosaics[i])[52:65]}_{onetile_obj_list[j]}.tiff"
                    cutout_path = os.path.join(output_dir, cutout_filename)
            
                
                img = cutout_data
                vmin = np.percentile(img, 0.5)  # Use lower percentile (1%) to include more faint features
                vmax = np.percentile(img, 99.5)  # Use 99th percentile to avoid extreme bright pixels
                img_stretched = np.clip(img, vmin, vmax)
                img_normalized = (img_stretched - vmin) / (vmax - vmin)
                img_new = Image.fromarray((img_normalized * 255).astype(np.uint8))
                # img_new = Image.fromarray(cutout_data)
    
                if not_defected(img_new):
                
                    img_new.save(cutout_path, format='TIFF') # save in output_dir directly 
            
                    # Add the file to the list of cutouts to be added to the ZIP file
                    cutout_files.append(cutout_filename)
    
    all_cutout_files.append(cutout_files) # append cutouts of each tile to cutout files for all tiles 
    if (i+1)%43 == 0:
        print(f'Completed: {(i+1)}/344')
    

/tmp/ipykernel_486568/56625887.py:60: RuntimeWarning: invalid value encountered in divide
  img_normalized = (img_stretched - vmin) / (vmax - vmin)
/tmp/ipykernel_486568/56625887.py:61: RuntimeWarning: invalid value encountered in cast
  img_new = Image.fromarray((img_normalized * 255).astype(np.uint8))


Completed: 43/344
Completed: 86/344
Completed: 129/344
Completed: 172/344
Completed: 215/344
Completed: 258/344
Completed: 301/344
Completed: 344/344
